# HAL

Hyperspace Analogue to Language (HAL) is an early vector space model (VSM) of semantics. VSMs did exist prior to HAL, but I gather that HAL was the first to construct the space automatically. Since Osgood et al. (1957), VSMs were built from human judgements on a predefined set of axes. Practically, this is a time-consuming process. It's also not a good model of the acquisition of semantics (although I doubt it's meant to be that).

This notebook implements HAL based on Lund and Burgess (1996). The original paper was hard to follow. The comments in [this implementation](https://github.com/fozziethebeat/S-Space/blob/master/src/main/java/edu/ucla/sspace/hal/HyperspaceAnalogueToLanguage.java) provided the help I needed.

The model is as follows. A word-word co-occurrence matrix $X$ is built. For $N$ words in the vocabulary, it will be an $N \times N$ matrix. For every word $w$ in the corpus, we look $m$ words to the left and $m$ to the right. Words appearing within this window (in either direction) are recorded with strength inversely proportional to the linear distance between the words. If word $j$ appears within $m$ words before $i$, $X_{ij}$ is increased. You can reduce the dimensionality of $X$ after training.

In [5]:
from itertools import chain, combinations
import pandas as pd
import nltk
from nltk.util import ngrams

In [6]:
mycorpus = 'The horse raced past the barn fell .'.split()
mycorpus = [word.lower() for word in mycorpus]

In [7]:
def skipgrams(sequence, n, k):
    """Modified from NLTK to give distance."""
    
    SENTINEL = object()
    for ngram in ngrams(sequence, n + k, pad_right=True, right_pad_symbol=SENTINEL):
        head = ngram[:1]
        tail = ngram[1:]
        for i, skip_tail in enumerate(combinations(tail, n - 1)):
            if skip_tail[-1] is SENTINEL:
                continue
            yield (head + skip_tail, k-i)

In [8]:
def hal(corpus, window):
    skips = skipgrams(corpus, 2, window)
    X = pd.DataFrame(skips, columns=['skipgram', 'weight'])
    X[['word1', 'word2']] = X['skipgram'].apply(pd.Series)
    X.drop('skipgram', axis=1, inplace=True)
    X = X.groupby(['word1', 'word2']).sum().unstack().fillna(0).astype(int)
    X.columns = X.columns.levels[1].values
    X.index = X.index.values
    return X.T

In [9]:
hal(mycorpus, 5)

,barn,fell,horse,past,raced,the
.,4,5,0,2,1,3
barn,0,0,2,4,3,6
fell,5,0,1,3,2,4
horse,0,0,0,0,0,5
past,0,0,4,0,5,3
raced,0,0,5,0,0,4
the,0,0,3,5,4,2


### Data

In [6]:
brown = nltk.corpus.brown.words()
brown = list(brown)[:1000]

In [8]:
X = hal(brown, 5)

In [9]:
X

,'',(,),",",.,1,13,1913,1923,1937,...,who,widespread,wife,wife's,will,with,won,work,would,year
'',0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
(,4,0,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
),2,8,0,0,3,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
",",26,0,0,12,8,0,0,0,0,0,...,0,1,6,0,0,0,0,0,0,0
.,62,0,0,15,0,5,0,5,5,5,...,0,0,0,0,1,0,0,0,1,5
1,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1913,0,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1937,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
